In [1]:
import sys
print (sys.version)
import pandas as pd
import quandl as qd
import numpy as np
import math
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
%matplotlib
import matplotlib.pyplot as plt
import datetime

3.5.2 |Anaconda 4.2.0 (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]
Using matplotlib backend: Qt5Agg


In [2]:
df = qd.get('WIKI/GOOGL')
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0


In [3]:
df = df[['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume']]
df.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0


In [4]:
df['HL_PCT'] = (df['Adj. High']-df['Adj. Close'])/df['Adj. Close']*100.0
df['PCT_change'] = (df['Adj. Close']-df['Adj. Open'])/df['Adj. Open']*100.0
df.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,HL_PCT,PCT_change
Date,,,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0,3.712563,0.324968
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0,0.710922,7.227007
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0,3.729433,-1.227880
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0,6.417469,-5.726357
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0,1.886792,1.183658


In [5]:
df = df[['Adj. Close','HL_PCT','PCT_change','Adj. Volume']]
df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0
2004-08-20,54.322689,0.710922,7.227007,22834300.0
2004-08-23,54.869377,3.729433,-1.227880,18256100.0
2004-08-24,52.597363,6.417469,-5.726357,15247300.0
2004-08-25,53.164113,1.886792,1.183658,9188600.0


Be aware about bias classifier!!

We have just create features, now Labels

In [6]:
forecast_col = 'Adj. Close'

In [7]:
df.fillna(-99999,inplace=True)

## Regression algorithm

In [8]:
forecast_out = int(math.ceil(0.01*len(df)))
forecast_out

33

In [9]:
forecast_col

'Adj. Close'

Now we need label

In [10]:
df['label'] = df[forecast_col].shift(-forecast_out)

In [11]:
df.dropna(inplace=True)
df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0,68.752232
2004-08-20,54.322689,0.710922,7.227007,22834300.0,69.639972
2004-08-23,54.869377,3.729433,-1.227880,18256100.0,69.078238
2004-08-24,52.597363,6.417469,-5.726357,15247300.0,67.839414
2004-08-25,53.164113,1.886792,1.183658,9188600.0,68.912727


In [12]:
X = np.array(df.drop(['label'],1))
Y = np.array(df['label'])

In [13]:
X = preprocessing.scale(X)

C:\Users\Jacopo\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [14]:
Y = np.array(df['label'])
len(X),len(Y)

(3173, 3173)

Let's test it

In [15]:
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X,Y,test_size=0.2)

classifier

In [16]:
clf = LinearRegression()
clf.fit(X_train,Y_train)
accuracy = clf.score(X_test,Y_test)
accuracy

0.97181145414963499

In [17]:
clf = svm.SVR(kernel='poly') # vector machine
clf.fit(X_train,Y_train)
accuracy = clf.score(X_test,Y_test)
accuracy

0.72516884326468289

run parallel

In [18]:
clf = LinearRegression(n_jobs=10)
clf.fit(X_train,Y_train)
accuracy = clf.score(X_test,Y_test)
accuracy

0.97181145414963499

In [19]:
clf = LinearRegression(n_jobs=-1)
clf.fit(X_train,Y_train)
accuracy = clf.score(X_test,Y_test)
accuracy

0.97181145414963499

## Regression forecasting and predicting - Practical Machine Learning Tutorial with Python p.5

In [20]:
df['label'] = df[forecast_col].shift(-forecast_out)
X = np.array(df.drop(['label'],1))
X = preprocessing.scale(X)
X_lately = X[-forecast_out:]
X = X[:-forecast_out]

df.dropna(inplace=True)
Y = np.array(df['label'])


X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X,Y,test_size=0.2)

clf = LinearRegression(n_jobs=-1)
clf.fit(X_train,Y_train)
accuracy = clf.score(X_test,Y_test)
accuracy

C:\Users\Jacopo\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


0.97074541073212484

## Let's predict the next 33 days
X are the features

Y are the labels

In [21]:
forecast_set = clf.predict(X_lately)
forecast_set,accuracy,forecast_out

(array([ 837.88819555,  838.31841681,  842.83932509,  846.96103072,
         848.07529125,  845.26634107,  850.371313  ,  854.4671294 ,
         857.20054934,  859.26763451,  858.98601129,  855.90545056,
         857.75037778,  852.87267004,  864.36069704,  857.74440508,
         857.24568507,  855.42685804,  859.14921495,  861.64469856,
         865.59025231,  869.28942215,  872.39962111,  873.87267248,
         876.26531988,  877.91688858,  879.97338742,  875.68272742,
         856.65245475,  857.52496712,  846.84032911,  842.59454488,
         845.94047587]), 0.97074541073212484, 33)

In [22]:
df['Forecast'] = np.nan
last_day = df.iloc[-1].name
last_day

Timestamp('2017-02-07 00:00:00')

In [23]:
last_unix = last_day.timestamp()
oneDay = 24*60*60 # seconds in a day
next_unix = last_unix + oneDay

## Let's populate

In [28]:
for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += oneDay
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)] + [i]

https://youtu.be/QLVMqwpOLPk?t=9m26s

http://stackoverflow.com/questions/31593201/pandas-iloc-vs-ix-vs-loc-explanation